### 1. 라이브러리 불러오기

In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, split, regexp_replace, when, length

### 2. 세션 생성

In [2]:
spark = SparkSession.builder \
    .appName("medi_test") \
    .getOrCreate()

23/11/27 15:27:18 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. 환경 변수 설정

In [3]:
root_path = '/Users/b06/Desktop/yeardream/medi-05'
json_root_path = f'{root_path}/data/naverplace_meta'
save_root_path = f'{root_path}/spark-scala-project/output/pyspark'
text_root_path = f'{root_path}/spark-scala-project/test.txt'

### 4. 데이터 불러오기

In [4]:
def read_text():
    with open(text_root_path, 'r') as t: 
        l = t.readlines()        
    n = l.pop(0).strip()    
    with open(text_root_path, 'w') as t: 
        t.writelines(l)
    return n

In [5]:
def read_json(n):
    json_path = f'{json_root_path}/naverplace_meta_{n}.json'
    data = spark.read.json(json_path)
    return data

In [6]:
# n = read_text()
n = 1
data = read_json(n)

23/11/27 15:27:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


### 5. Columns & Schema

In [7]:
columns = data.columns
hospital_bases = [c for c in columns if "HospitalBase" in c]

In [8]:
df_schema = StructType([
    StructField('id', StringType(), True),
    StructField('name', StringType(), True),
    StructField('review_keywords', StringType(), True),
    StructField('description', StringType(), True),
    StructField('road', StringType(), True),
    StructField('booking_business_id', StringType(), True),
    StructField('booking_display_name', StringType(), True),
    StructField('category', StringType(), True),
    StructField('category_code', StringType(), True),
    StructField('category_code_list', StringType(), True),
    StructField('category_count', StringType(), True),
    StructField('rcode', StringType(), True),
    StructField('virtual_phone', StringType(), True),
    StructField('phone', StringType(), True),
    StructField('naver_booking_url', StringType(), True),
    StructField('conveniences', StringType(), True),
    StructField('talktalk_url', StringType(), True),
    StructField('road_address', StringType(), True),
    StructField('keywords', StringType(), True),
    StructField('payment_info', StringType(), True),
    StructField('ref', StringType(), True),
    StructField('lon', StringType(), True),
    StructField('lat', StringType(), True) 
])
df = spark.createDataFrame([], df_schema)

### 6. 함수

In [9]:
def get_value(data, base_id, key):
    column_key = f'HospitalBase:{base_id}.{key}'
    column = data.select(column_key)
    row = column.filter(~col(key).isNull())
    value = row.first()
    return value

def replace_expr_and_get_value(value):
    if value:
        value = value \
            .replace("\n", "") \
            .replace("\r", "") \
            .replace(",", "") \
            .replace("*", "")
        return value
    else:
        return None
    
def check_none(value):
    if value:
        return value[0]
    else:
        return None
    
def get_lon_lat_values(ref_value):
    if ref_value:
        panorama_key = ref_value.split(":")[1]
        panorama_data = data.selectExpr(f"`Panorama:{panorama_key}` as panorama_data")
        panorama_value = panorama_data.filter(~col('panorama_data').isNull()).first()
        if panorama_value:
            lon_lat_values = {
                'lon': float(panorama_value[0]['lon']),
                'lat': float(panorama_value[0]['lat'])
            }
        return lon_lat_values
    else:
        return None

In [10]:
def create_boolean_column(df, target_column, create_column, value = None):
    if value is not None:
        cond = col(target_column).isNotNull() & col(target_column).contains(value)
    else:
        cond = col(target_column).isNotNull() & (col(target_column) != "")
    df = df.withColumn(create_column, when(cond, True).otherwise(False))
    return df

def expand_column(df, column, max_num):
    create_array = f'{column}_array'
    df = df.withColumn(create_array, split(col(column), ','))
    for i in range(max_num):
        df = df.withColumn(f'{column}_{i+1}', col(create_array)[i])
    return df

def replace_expr_in_keyword_columns(df):
    for i in range(1, 6):
        keyword_column = f'keywords_{i}'
        df = df.withColumn(
            keyword_column, 
            when(col(keyword_column).isNotNull(),
                 regexp_replace(col(keyword_column), "[\[\]]", ""))
        )
    return df

### 7. 데이터 추출

In [11]:
hospital_data = []
for hospital_base, base_id in zip(hospital_bases, [hospital_base.split(":")[1].strip() for hospital_base in hospital_bases]):
    # get values
    id_value = get_value(data, base_id, 'id')
    name_value = get_value(data, base_id, 'name')
    review_keywords_value = get_value(data, base_id, 'reviewSettings')
    description_value = get_value(data, base_id, 'description')
    road_value = get_value(data, base_id, 'road')
    bookingBusinessId_value = get_value(data, base_id, 'bookingBusinessId')
    bookingDisplayName_value = get_value(data, base_id, 'bookingDisplayName')
    category_value = get_value(data, base_id, 'category')
    categoryCode_value = get_value(data, base_id, 'categoryCode')
    categoryCodeList_value = get_value(data, base_id, 'categoryCodeList')
    categoryCount_value = get_value(data, base_id, 'categoryCount')
    rcode_value = get_value(data, base_id, 'rcode')
    virtualPhone_value = get_value(data, base_id, 'virtualPhone')
    phone_value = get_value(data, base_id, 'phone')
    naverBookingUrl_value = get_value(data, base_id, 'naverBookingUrl')
    conveniences_value = get_value(data, base_id, 'conveniences')
    talktalkUrl_value = get_value(data, base_id, 'talktalkUrl')
    roadAddress_value = get_value(data, base_id, 'roadAddress')
    keywords_value = get_value(data, base_id, 'keywords')
    paymentInfo_value = get_value(data, base_id, 'paymentInfo')
    streetPanorama_value = get_value(data, base_id, 'streetPanorama')
    print(f"got HospitalBase:{base_id}'s values")

    # check none
    id_value = check_none(id_value)
    name_value = check_none(name_value)
    review_keywords_value = review_keywords_value[0]['keyword'] if review_keywords_value else None
    description_value = check_none(description_value)
    road_value = check_none(road_value)
    bookingBusinessId_value = check_none(bookingBusinessId_value)
    bookingDisplayName_value = check_none(bookingDisplayName_value)
    category_value = check_none(category_value)
    categoryCode_value = check_none(categoryCode_value)
    categoryCodeList_value = check_none(categoryCodeList_value)
    categoryCount_value = check_none(categoryCount_value)
    rcode_value = check_none(rcode_value)
    virtualPhone_value = check_none(virtualPhone_value)
    phone_value = check_none(phone_value)
    naverBookingUrl_value = check_none(naverBookingUrl_value)
    conveniences_value = check_none(conveniences_value)
    talktalkUrl_value = check_none(talktalkUrl_value)
    roadAddress_value = check_none(roadAddress_value)
    keywords_value = check_none(keywords_value)
    paymentInfo_value = check_none(paymentInfo_value)
    ref_value = streetPanorama_value[0]['__ref'] if streetPanorama_value else None
    lon_lat_values = get_lon_lat_values(ref_value)
    print(f"checked HospitalBase:{base_id}'s values")
    
    # Replace expressions and get values
    road_value = replace_expr_and_get_value(road_value)
    description_value = replace_expr_and_get_value(description_value)
    review_keywords_value = None if review_keywords_value is None else review_keywords_value.replace("\\", "").replace("\"", "")
    print(f"replaced HospitalBase:{base_id}'s expressions")
    
    # create rows
    rows = Row(
        id=base_id,
        name=name_value,
        review_keywords=review_keywords_value,
        description=description_value,
        road=road_value,
        booking_business_id=bookingBusinessId_value,
        booking_display_name=bookingDisplayName_value,
        category=category_value,
        category_code=categoryCode_value,
        category_code_list=categoryCodeList_value,
        category_count=categoryCount_value,
        rcode=rcode_value,
        virtual_phone=virtualPhone_value,
        phone=phone_value,
        naver_booking_url=naverBookingUrl_value,
        conveniences=conveniences_value,
        talktalk_url=talktalkUrl_value,
        road_address=roadAddress_value,
        keywords=keywords_value,
        payment_info=paymentInfo_value,
        ref=ref_value,
        lon=lon_lat_values['lon'] if lon_lat_values else None,
        lat=lon_lat_values['lat'] if lon_lat_values else None
    )
    hospital_data.append(rows)
    print(f"appended HospitalBase:{base_id}'s rows\n")

got HospitalBase:1049624743's values
checked HospitalBase:1049624743's values
replaced HospitalBase:1049624743's expressions
appended HospitalBase:1049624743's rows

got HospitalBase:1094819082's values
checked HospitalBase:1094819082's values
replaced HospitalBase:1094819082's expressions
appended HospitalBase:1094819082's rows

got HospitalBase:11779766's values
checked HospitalBase:11779766's values
replaced HospitalBase:11779766's expressions
appended HospitalBase:11779766's rows

got HospitalBase:1233961055's values
checked HospitalBase:1233961055's values
replaced HospitalBase:1233961055's expressions
appended HospitalBase:1233961055's rows

got HospitalBase:12787598's values
checked HospitalBase:12787598's values
replaced HospitalBase:12787598's expressions
appended HospitalBase:12787598's rows

got HospitalBase:12857046's values
checked HospitalBase:12857046's values
replaced HospitalBase:12857046's expressions
appended HospitalBase:12857046's rows

got HospitalBase:1298196933'

In [12]:
# create dataframe from df_schema
df = spark.createDataFrame(hospital_data, schema=df_schema)

### 8. 데이터 전처리

In [13]:
# deduplicate
# select columns
df = df.dropDuplicates([
    "id",
    "name",
    "review_keywords",
    "description",
    "road",
    "booking_business_id",
    "booking_display_name",
    "category",
    "category_code",
    "category_code_list",
    "category_count",
    "rcode",
    "virtual_phone",
    "phone",
    "naver_booking_url",
    "conveniences",
    "talktalk_url",
    "road_address",
    "keywords",
    "payment_info",
    "lon",
    "lat"
])

In [14]:
df = df.withColumn('description_length', length('description'))
df = create_boolean_column(df, 'virtual_phone', 'is_virtual_phone')
df = create_boolean_column(df, 'payment_info', 'zero_pay', '제로페이')
df = expand_column(df, 'keywords', 5)
df = replace_expr_in_keyword_columns(df)

In [15]:
hospital_df=df.select(
    "id",
    "name",
    "review_keywords",
    "description",
    "description_length",
    "road",
    "booking_business_id",
    "booking_display_name",
    "category",
    "category_code",
    "category_code_list",
    "category_count",
    "rcode",
    "virtual_phone",
    "is_virtual_phone",
    "phone",
    "naver_booking_url",
    "conveniences",
    "talktalk_url",
    "road_address",
    "keywords_1",
    "keywords_2",
    "keywords_3",
    "keywords_4",
    "keywords_5",
    "payment_info",
    "zero_pay",
    "lon",
    "lat"
)

### 9. 데이터 저장

In [19]:
import shutil
test_save_path = f'{save_root_path}/test/hospital_df'
shutil.rmtree(test_save_path)

In [20]:
# save csv from data    
def save_to_csv(df, name):
    save_path = f'{save_root_path}/test/{name}'
    df \
        .coalesce(1) \
        .write \
        .mode('append') \
        .option("encoding", "utf-8") \
        .csv(save_path, header=True)    

In [21]:
save_to_csv(hospital_df, "hospital_df")

### 10. 데이터 확인

In [22]:
import pandas as pd
import os

In [23]:
save_dir_list = os.listdir(test_save_path)
save_name = [f for f in save_dir_list if f.endswith('.csv')][0]
save_path = f'{save_root_path}/test/hospital_df/{save_name}'
print_df = pd.read_csv(save_path, encoding='utf-8')
print_df.head(10)

,id,name,review_keywords,description,description_length,road,booking_business_id,booking_display_name,category,category_code,...,road_address,keywords_1,keywords_2,keywords_3,keywords_4,keywords_5,payment_info,zero_pay,lon,lat
0,265615453,본디올신촌경희한의원,(본디올신촌경희한의원) & (서대문구 | 북아현동 | 한의원-일반) & 본디올신촌경...,본디올신촌경희한의원은 지하철 2호선 아현역 1번 출구 앞에서 1분 거리에 있으며 통...,97.0,지하철 이용 : 아현역 1번 출구로 나와서 오른쪽으로 보이는 상가로 끝까지 들어오시...,224094.0,방문,한의원,223265,...,서울 서대문구 신촌로35길 10,아현역한의원,아현동한의원,아현동체질한의원,아현역교통사고한의원,아현역1번출구한의원,[제로페이],True,126.956469,37.558709
1,1094819082,큰나무한의원,(큰나무한의원) & (마포구 | 공덕동 | 한의원-일반) & 큰나무한의원,NaN,NaN,NaN,NaN,방문,한의원,223265,...,서울 마포구 마포대로 143,NaN,NaN,NaN,NaN,NaN,NaN,False,126.953424,37.547404
2,11779766,다인한의원,(다인한의원) & (용산구 | 청파동2가 | 한의원-일반) & 다인한의원,숙대 앞 청파로에 위치한 한방재활의학과 전문의 한의학 박사가 진료하는 한의원입니다....,231.0,지하철 숙대입구역 10번에서 뒤로 돌아 숙대 방향으로 500M 올라오시거나 400번...,487661.0,방문,한의원,223265,...,서울 용산구 청파로47길 66 중하빌딩 2층,통증퇴행성질환,다이어트비만,교통사고,피부,생리불순,[제로페이],True,126.966683,37.545033
3,1049624743,경희닥터권한의원,(경희닥터권한의원) & (마포구 | 공덕동 | 한의원-일반) & 경희닥터권한의원,현재 휴진중이며 이전개원 예정입니다.한의학 박사 권선근 원장이 진료하는 경희닥터권 ...,258.0,공덕역 3번출구로 나오셔서 300m 직진버스 이용시 (아현동 주민센터 정류장 하차)...,NaN,방문,한의원,223265,...,서울 마포구 마포대로 143 파크팰리스2 209호,ADHD,소아한의원,성장,비염,틱,NaN,False,126.953360,37.547317
4,1233961055,원광좋은한의원,(원광좋은한의원) & (마포구 | 공덕동 | 한의원-일반) & 원광좋은한의원,편안하고 아늑한 분위기에서 최선의 진료로 만족감을 드리겠습니다.저희 한의원은 진료실...,229.0,공덕역 4번출구 신한은행(지방재정회관) 우회전 GS25 편의점 건물(영명빌딩) 3층...,NaN,방문,한의원,223265,...,서울 마포구 마포대로8길 9 영명빌딩 3층,공덕_한의원,다이어트,공진단,자동차사고,약침,"[지역화폐(지류,카드,모바일), 제로페이]",True,126.954072,37.546583
5,12787598,서울한의원,서울한의원,몸과 마음 그리고 생활의 건강한 변화!! 서울 한의원은 당신의 건강 지킴이 입니다....,219.0,지하철: 5 6호선 공덕역 5번 출구로 나와서 20 m 직진 (하얀색 제일빌딩 2층...,NaN,방문,한의원,223265,...,서울 마포구 만리재로 15 제일빌딩,NaN,NaN,NaN,NaN,NaN,NaN,False,126.952681,37.544300
6,12857046,동제한의원,(동제한의원) & (마포구 | 아현동 | 한의원-일반) & 동제한의원,NaN,NaN,NaN,NaN,방문,한의원,223265,...,서울 마포구 굴레방로 27 보성상가 301호,NaN,NaN,NaN,NaN,NaN,[제로페이],True,126.955585,37.556398
7,1298196933,동경한의원 용산역점,(동경한의원 용산역점) & (용산구 | 한강로2가 | 한의원-일반) & 동경한의원 ...,[자반증한의원 혈관염한의원 방문 전 필독]안녕하세요. 14년째 자반증/혈관염에 집중...,1093.0,지하철 4호선 신용산역 3번출구 지하 연결 지하철 1호선 / 경의선 용산역 1번출구...,265408.0,방문,한의원,223265,...,서울 용산구 한강대로 95 B동 2층 220호,자반증,한방다이어트,색소성자반증,알레르기성자반증,청피반성혈관염,NaN,False,126.967814,37.529719
8,1344046290,수풀림한의원,(수풀림한의원) & (용산구 | 한강로1가 | 한의원-일반) & 수풀림한의원,바른 철학으로 치료하는 수풀림한의원 진료과목1. 자가면역질환류마티스관절염 및 기타 ...,529.0,수풀림한의원은 4 6호선 삼각지역 5번 출구 바로 앞에 위치해있습니다.차량을 가져오...,988713.0,방문,한의원,223265,...,서울 용산구 한강대로 171 1층 수풀림한의원,용산한의원,다이어트한약,류마티스관절염,섬유근육통,한방다이어트,"[지역화폐(카드), 제로페이]",True,126.972661,37.534331
9,13463823,청파약손한의원,(청파약손한의원) & (용산구 | 서계동 | 한의원-일반) & 청파약손한의원,사상의학을 바탕으로 보다 나은 치료를 위해 항상 노력하는 청파약손한의원입니다.늘어진...,205.0,서울역 15번 출구(예전 서부역)에서 남영역 방향으로 200미터 오시면 됩니다,NaN,방문,한의원,223265,...,서울 용산구 청파로 349 102호,비만클리닉,추나요법,자동차보험,서울역한의원,디스크관절질환,"[지역화폐(지류,카드,모바일), 제로페이]",True,126.969567,37.550832
